### MarianTokenizer 사용해보기
> Transformer 구현에 사용된 토크나이저

In [1]:
from transformers import MarianMTModel, MarianTokenizer 
# MT: Machine Translation

# Load the tokenizer & model
tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ko-en')
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ko-en') # MT: Machine Translation

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


#### 필수적인 Token
- `eos`: 문장의 시작과 끝을 알리는 토큰
- `pad`: Batch 의 Size 를 맞춰주기 위해 부족한 길이를 채우는 토큰

In [2]:
# 필수적인 token 의 ID
eos_idx = tokenizer.eos_token_id
pad_idx = tokenizer.pad_token_id
print('eos_idx =', eos_idx)
print('pad_idx =', pad_idx)

# 토크나이저의 어휘 사전 크기를 가져옴
vocab_size = tokenizer.vocab_size
print("Vocabulary size:",vocab_size)

# 토크나이저의 어휘 사전
# print(tokenizer.get_vocab())

eos_idx = 0
pad_idx = 65000
Vocabulary size: 65001


#### 토크나이저 & 학습된 모델 사용

- [토크나이저 참고 자료 - Byte Pair Encoding](https://ratsgo.github.io/nlpbook/docs/preprocess/bpe/)

In [3]:
# _ 는 띄어쓰기를 의미하며, 단어, 음절이 아닌 Subword tokenizing 방식
print(tokenizer.tokenize("토크나이저"))
print(tokenizer.tokenize("요새는 Claude 가 더 좋은 듯"))
print(tokenizer.tokenize("한글에서 제일 좋은 tokenizer 는 뭘까"))

['▁토', '크', '나이', '저']
['▁요새', '는', '▁Cl', 'au', 'de', '▁가', '▁더', '▁좋은', '▁듯']
['▁한', '글', '에서', '▁제일', '▁좋은', '▁to', 'k', 'en', 'iz', 'er', '▁는', '▁뭘', '까']


In [7]:
# word to index
print(tokenizer.encode('인공', add_special_tokens=False)) 
print(tokenizer.encode('지능', add_special_tokens=False)) 
print(tokenizer.encode('<pad>', add_special_tokens=False)) 
print(tokenizer.encode('</s>', add_special_tokens=False))
print('-' * 10)

# Sentence to index
print(tokenizer.tokenize("문장을 넣으면 토크나이즈를 하여 숫자로 바꾼다"))
print(tokenizer.encode('문장을 넣으면 토크나이즈를 하여 숫자로 바꾼다', add_special_tokens=False)) 

# index to string
print(tokenizer.decode([204]))
print(tokenizer.decode([206]))
print(tokenizer.decode([210]))
print(tokenizer.decode(list(range(5,9)) + [65000,65001,65002,65003]))

[19025]
[34359]
[65000]
[0]
----------
['▁문장', '을', '▁넣으면', '▁토', '크', '나이', '즈', '를', '▁하여', '▁숫자', '로', '▁바꾼', '다']
[13774, 51, 40068, 4155, 1020, 5037, 1329, 79, 242, 6635, 131, 30737, 161]
사람
으로
make
to of? and<pad> <unk> <unk> <unk>


In [10]:
# 사전 학습된 모델로 번역해보기
input_text = "토크나이저는 문장을 숫자로 바꾼다"

input_tokens = tokenizer.encode(input_text, return_tensors="pt")
translated_tokens = model.generate(input_tokens)
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

print("입력:", input_text)
print("AI의 번역:", translated_text)

입력: 토크나이저는 문장을 숫자로 바꾼다
AI의 번역: Toknaiser turns sentences into numbers.


### Dataset, DataLoader 생성 및 배치단위 토크나이징

> [AI hub](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=126)

In [11]:
import pandas as pd
import torch

!gdown https://drive.google.com/uc?id=1r4ZnFJOStyBlNRx7snBQ-Iq2GNyJKL6t -O '대화체.xlsx'
data = pd.read_excel('대화체.xlsx')

Downloading...
From: https://drive.google.com/uc?id=1r4ZnFJOStyBlNRx7snBQ-Iq2GNyJKL6t
To: /root/transformer/대화체.xlsx
100%|██████████████████████████████████████| 9.57M/9.57M [00:00<00:00, 27.8MB/s]


In [13]:
data.head(1)

,대분류,소분류,상황,Set Nr.,발화자,원문,번역문
0,비즈니스,회의,의견 교환하기,1,A-1,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...


In [14]:
import torch.utils.data


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data) -> None:
        self.data = data
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        return self.data.loc[index, '원문'], self.data.loc[index, '번역문']
    
custom_DS = CustomDataset(data=data)

train_DS, val_DS, test_DS= torch.utils.data.random_split(custom_DS, [95000, 3000, 2000])

BATCH_SIZE = 64
train_DL = torch.utils.data.DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
val_DL = torch.utils.data.DataLoader(val_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = torch.utils.data.DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=True)

print(len(train_DS))
print(len(val_DS))
print(len(test_DS))

95000
3000
2000


In [16]:
max_len = 100
for src_texts, trg_texts in train_DL:
    
    print(len(src_texts), len(trg_texts))
    print(src_texts[:2])
    print( trg_texts[:2])
    
    print('-----Tokenizing-----')
    # truncation = True: max_len 보다 길면 끊고 <eos> 를 추가한다.
    # pt: pytorch tensor로 변환
    src = tokenizer(src_texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt').input_ids
    # target 의 첫 입력 단어로 <sos> 를 추가한다.
    trg_texts = ['</s> ' + s for s in trg_texts]
    trg = tokenizer(trg_texts, padding=True, truncation=True, max_length = max_len, return_tensors='pt').input_ids
    
    # Batch 의 내 포함된 문장의 최대 길이보다 
    # 짧은 문장은 <pad> 로 채워지게 된다.
    print(src.shape, trg.shape)
    print(src[:2])
    
    break

64 64
('그건 진짜 무례한데 어떻게 사과를 안 할 수가 있지?', '입국심사장 가는 길에 제 여권이 없다는 걸 발견했어요. 잃어버린 것 같아요.')
('How can someone not apologize for such a rude act?', "I realized that I don't have my passport on the way to immigration. I think I lost it.")
-----Tokenizing-----
torch.Size([64, 27]) torch.Size([64, 48])
tensor([[  995,  1670, 28049, 13210,   333,  7980,    79,   245,   239,  3211,
          1695,     7,     0, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000],
        [ 2341,  1815,   865, 20095,  3222, 17144,   303, 43359,    48,  6956,
           637, 45786,     2, 29154,   346,  3300,     2,     0, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000]])
